In [1]:
import os, sys
import re
import numpy as np
import imageio as im
import pandas as pd
import cv2
from skimage.morphology import skeletonize
from skimage.util import invert
from IPython.display import Image
import matplotlib.pyplot as plt
from cv2 import imwrite
from cv_algorithms import grassfire
from cv_algorithms import thinning as ta
from cv_algorithms.morphology import difference_of_gaussian
from fil_finder import FilFinder2D
from astropy.io import fits
import astropy.units as u
from tools import fskel
from cellpose import utils
from IPython.core.display import HTML
from scipy.interpolate import splprep
from scipy.interpolate import splev
from tools import intersection
from tools import skeleton_to_centerline

cell_path = "individual_cells/"

In [167]:
#Helper functions for this notebook. Have not yet been added to tools
def intersection(line1,line2,width1=3,width2=3):
    '''
    Find if two lines intersect, or nearly intersect
    
    Parameteres
    -----------
    line1,line2=boolean arrays with 'True' values where the line exists
    width1,width2=amount (in pixels) the line should be dilated to see if a near intersection occurs
    '''
    m=np.shape(line1)[1]
    n=np.shape(line1)[0]
    d=m//2
    l_check = np.sum(line1[0:n,0:d]+line2[0:n,0:d]==2)>0
    r_check = np.sum(line1[0:n,d:m-1]+line2[0:n,d:m-1]==2)>0
    if l_check and r_check:
        return True
    else:
        kernel1 = cv2.getStructuringElement(cv2.MORPH_RECT,(width1,width1))
        kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT,(width2,width2))
        dilated1 = cv2.dilate(line1.astype(np.uint8),kernel1,iterations=1)
        dilated2 = cv2.dilate(line2.astype(np.uint8),kernel2,iterations=1)
        m=np.shape(line1)[1]
        n=np.shape(line1)[0]
        d=m//2
        l_check = np.sum(dilated1[0:n,0:d]+dilated2[0:n,0:d]==2)>0
        r_check = np.sum(dilated1[0:n,d:m-1]+dilated2[0:n,d:m-1]==2)>0
        if l_check and r_check:
            return True
        else:
            return False
def skeleton_to_centerline(skeleton):
    '''
    Finds an ordered list of points which trace out a skeleton with no branches by following the pixels of the centerline 
    elementwise. Starts at the point farthest left and closest to the top. Ends when there are no new points on the skeleton 
    to go to.
    Parameters
    ----------
    skel = a 2d numpy boolean array representing a topological skeleton with no branches
    '''
    #initializing lists and starting parameters
    centerline=[]
    pos = bool_sort(skeleton)[0] # finds and sorts all points where the skeleton exists. Selects leftest as initial position
    centerline.append(pos) #appends initial position to the centerline
    skeleton[pos[0],pos[1]]=False #erases the previous positon
    local = make_local(skeleton,pos)
    while np.any(local): #checks if there are any new positions to go to
        pos = pos+bool_sort(local)[0]-np.array([1,1]) #updates position
        centerline.append(pos) #adds position to centerline
        skeleton[pos[0],pos[1]]=False #erases previous position
        local = make_local(skeleton,pos) #updates the local minor for the new position
    return centerline
def make_local(matrix,pos):
    '''
    Extracts the 3x3 array of values around a position in an existing array. If this is not possible (if the position 
    is along an edge), completes the array with rows/columns of 'False' values
    Paramters
    ---------
    matrix = a 2d numpy array
    pos = a numpy array or list, indicating a position in array
    '''
    N = np.shape(matrix)[0]+1
    M = np.shape(matrix)[1]+1
    n1 = max(0,pos[0]-1)
    n2 = min(N,pos[0]+2)
    m1 = max(0,pos[1]-1)
    m2 = min(M,pos[1]+2)
    if n1 == pos[0]-1 and n2 != N and m1==pos[1]-1 and m2!=M:
        return matrix[n1:n2,m1:m2]
    elif n1 == 0:
        if m1 == 0:
            return np.hstack((np.array([[False],[False],[False]]),np.vstack((np.array([False,False]), matrix[n1:n2,m1:m2]))))
        elif m2 == M:
            return np.hstack((np.vstack((np.array([False,False]), matrix[n1:n2,m1:m2])),np.array([[False],[False],[False]])))
        else:
            return np.vstack((np.array([False,False,False]), matrix[n1:n2,m1:m2]))
    elif n2 == N:
        if m1 == 0:
            return np.hstack((np.array([[False],[False],[False]]),np.vstack((matrix[n1:n2,m1:m2],np.array([False,False])))))
        elif m2==M:
            return np.hstack((np.vstack((matrix[n1:n2,m1:m2],np.array([False,False]))),np.array([[False],[False],[False]])))
        else:
            return np.vstack((matrix[n1:n2,m1:m2],np.array([False,False,False])))
    elif m1==0:
        return np.hstack((np.array([[False],[False],[False]]),matrix[n1:n2,m1:m2]))
    elif m2 ==M:
        return np.hstack((matrix[n1:n2,m1:m2],np.array([[False],[False],[False]])))
def bool_sort(array):
    '''
    Sort the "True" values in a boolean array, starting from left to right, then top to bottom
    Parameters
    ---------
    array = a boolean array to be sorted
    '''
    output = np.transpose(np.array(list(np.where(array))))
    output = output[output[:,0].argsort()]
    output = output[output[:,1].argsort(kind='mergesort')]
    return output

In [28]:
all_the_cells = os.listdir(cell_path)

#a list of outlines of cell masks with timepoints and ids
outl = []
# List of masks with associated timepoints and names
maskl = []
#list of pruned skeletons with associated timepoints
skell = []
#list of centerlines extracted from extracted from skeletons, with associated timepoints
centerl=[]

for cell in all_the_cells:
    time = re.findall(r'\d+', cell)
    if time != []:
        time = time[0]
        im_path = cell_path + cell
        for mask_name in os.listdir(im_path + '\masks'):
            mask=im.imread(im_path + '\masks'+'/'+mask_name)[:,:,0]>0 #retrive the cells mask (boolean array)
            skel = im.imread(im_path + '\skeletons' + '\pruned_' + mask_name[5:6] +'.png')>0
            out = utils.masks_to_outlines(mask) #create the cells outline (boolean array)
            inter = out + skel ==2 #find the intersection of the outline as a boolean array
            outl.append([out,int(time),int(mask_name[5:6])])
            maskl.append([mask,int(time),int(mask_name[5:6])])
            width = max(np.shape(out))//30
            if intersection(out,skel,width,width): #if the intersection has more than two elements, include the skeleton as a complete skeleton
                skell.append([skel,int(time),int(mask_name[5:6]),True])
            else:
                skell.append([skel,int(time),int(mask_name[5:6]),False])

for skel in skell:
    [skel0,time,ID,complete] = skel
    skel0 = np.array(skel0)
    time = int(time)
    ID = int(ID)
    complete = bool(complete)
    center = skeleton_to_centerline(skel0)
    centerl.append([center,time,ID,complete])

In [29]:
splines = []
for center in centerl:
    if len(center[0])>5:
        tck,u=splprep(np.transpose(center[0]))
        time = int(center[1])
        ID = int(center[2])
        complete = bool(center[3])
        splines.append([tck,u,time,ID,complete])

In [252]:
for spline in splines:
    if spline[-1]:
        time = spline[2]
        ID = spline[3]
        skel = [s for s in skell if s[1]==time and s[2]==ID][0][0]
        mask = [m for m in maskl if m[1]==time and m[2]==ID][0][0]
        overlay = invert(skel + invert(mask))
        tck = spline[0]
        u = spline[1]
        [y,x] = splev(spline[1],spline[0])
        plt.imshow(overlay,cmap=plt.cm.gray)
        plt.plot(x,y,'r-')
        plt.savefig('complete splines' + '/' + str(time) + '_' +str(ID))
        plt.clf()

<Figure size 1800x1200 with 0 Axes>